# Channel Break Out Detection

## Import Data
##### 1. Determinar puntos de cambios de tendencia
##### 2. Con los valores high determinar la linea de tendencia bajista
##### 3. Determinar el punto de ruptura del canal, mas que todo la tendcia techo (canal bajista)
##### 4. Determinar manipulacion del mercado y falsa ruptura

In [1]:
import pandas as pd
import numpy as np
#import plotly.graph_objects as go
#import yfinance as yf
#import matplotlib.pyplot as plt
#import math
from datetime import date, datetime, timedelta
from scipy import stats
from scipy.signal import argrelextrema
#from bokeh.plotting import figure, show, column
from sklearn.linear_model import LinearRegression
import os
from backtesting import Backtest, Strategy

/usr/share/miniconda/envs/test/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
/usr/share/miniconda/envs/test/lib/python3.10/site-packages/backtesting/_plotting.py:55: UserWarning: Jupyter Notebook detected. Setting Bokeh output to notebook. This may not work in Jupyter clients without JavaScript support, such as old IDEs. Reset with `backtesting.set_bokeh_output(notebook=False)`.
  warnings.warn('Jupyter Notebook detected. '


Loading BokehJS ...

In [2]:
#df_h = pd.read_csv(r"D:\CARLOS\TRADING\data.txt")
df_h=pd.read_csv('data/dataxh.txt',sep='\t')
tickers = [
    'SPY',
    'META',
    'AAPL',
    'AMZN',
    'NFLX',
    'MRNA',
    'TSLA',
    'TNA',
    'GLD',
    'SLV',
    'USO',
    'BAC',
    'CVX',
    'XOM',
    'QQQ',
    'MSFT',
    'NVDA',
    'WMT',
    'BA',
    'DIS',
    'CAT',
    'IBM',
    'WFC',
    'PLTR',
    'AMD',
    'AVGO',
    'HOOD',
    'CRWV',
    'MSTR',
    'UNH',
    'GOOG',
    'APP',
    'UBER'
]

In [3]:
df_h['datetime'] = pd.to_datetime(df_h['datetime'])
df = pd.DataFrame()
for ticker in tickers:
    company = df_h.query("companyName==@ticker").copy()
    company.sort_values(by=['datetime'])
    #company = yf.download(ticker, start = ini2_str, end = today_str, interval='60m')
    #company.columns = [company.columns[0][0], company.columns[1][0], company.columns[2][0], company.columns[3][0], company.columns[4][0]]
    #company.rename(columns={'Datetime':'Gmt time'}, inplace = True)
    #company['datetime'] = pd.to_datetime (company.index)
    #company['companyName'] = ticker
    company['SMA20'] = company['Close'].rolling(20).mean()
    company.dropna(inplace=False)
    company['SMA40'] = company['Close'].rolling(40).mean()
    company.dropna(inplace=False)
    df = pd.concat([df, company],ignore_index=True)

## Detect Pivots/Fractals

In [4]:
ord=10
max_idx = argrelextrema(df['Close'].values, np.greater, order=ord)[0]
min_idx = argrelextrema(df['Close'].values, np.less, order=ord)[0]
df.loc[max_idx, 'pivotHigh'] = df['High']+1e-3
df.loc[min_idx, 'pivotLow'] = df['Low']-(1e-3)
df.loc[max_idx, 'isPivot'] = 1
df.loc[min_idx, 'isPivot'] = 2

In [5]:
#hoy =  np.datetime64('today')
#print(hoy)
#fecha_menos_15 = hoy -  np.timedelta64(15, 'D')
#print(fecha_menos_15)
#df2 = df[df["datetime"].dt.tz_localize(None)>=fecha_menos_15]

In [6]:
hoy =  np.datetime64('today')
fecha_menos_15 = hoy -  np.timedelta64(15, 'D')

###### Canales bajistas, cuando el PM de 40 esta por encima del PM de 20 en direccion desendente
###### RSI>70 y sobre las olas de Bollinger

In [7]:
df["datetime2"] = df["datetime"].dt.tz_localize(None)

In [8]:
def rcb01(df):
    hoy =  np.datetime64('today')
    fecha_menos_15 = hoy -  np.timedelta64(15, 'D')
    if ((df['pivotLow']>0) & (df['SMA40']>df['SMA20']) & (df["datetime2"]>=fecha_menos_15)):
        return 1
    else:
        return 0
df['rcb01_01'] = df.apply(rcb01, axis=1)

In [9]:
# Eliminar la columna 'datetime2'
df = df.drop('datetime2', axis=1)

## Detect Price Channel

In [10]:
def collect_channel(candle, backevalTrend, trendH):
    localdf = dfpl.loc[candle-backevalTrend:candle] #tomar en cuenta el backcandles

    highs = localdf.loc[localdf.index.isin(trendH)].High.values
    idxhighs = localdf.loc[localdf.index.isin(trendH)].High.index
    if len(highs)>=2:
        sl_highs, interc_highs, r_value_h, _, _ = stats.linregress(idxhighs,highs)    
        return(sl_highs, interc_highs, r_value_h**2)
    else:
        return(0,0,0)
    

In [11]:
def backcandle(candle):
    
    trendH = []
    backeval = 0
    val = 0
    
    dfpl['PM40_PM20'] = dfpl['SMA40']-dfpl['SMA20']

    if (dfpl.loc[candle,"PM40_PM20"]>0):
        idx_negativo = dfpl.loc[:candle, 'PM40_PM20'][::-1].lt(0).idxmax()
        backeval = idx_negativo - 4
        val = candle - backeval
        
    if val>75:
        backeval=candle-75
        
    if (dfpl.loc[backeval,"PM40_PM20"]>0):
        idx_negativo = dfpl.loc[:backeval, 'PM40_PM20'][::-1].lt(0).idxmax()
        backeval = idx_negativo - 4

    if val>75:
        backeval=candle-75

    ini = backeval
    trendprev=dfpl.loc[ini:candle]
    trend = trendprev.reset_index(drop=False)

    idxhighs = trend.index.to_numpy().reshape(-1,1) # Convertir X a una matriz 2D
    highs = trend.High.values
    modelo1 = LinearRegression()
    modelo1.fit(idxhighs, highs)
    # Obtener la predicción de la línea de regresión
    Y_pred = modelo1.predict(idxhighs)
    # Calcular la distancia solo de los puntos que están POR ENCIMA de la línea
    trend['Distancia'] = trend["High"] - Y_pred  # Diferencia entre el valor real y la predicción
    # Filtrar los puntos que están por encima (donde Y > Y_pred, es decir, Distancia > 0)
    df_encima = trend[trend['Distancia'] > 0]
    puntos_mas_lejanos = df_encima.nlargest(2, 'Distancia')   #2 puntos mas lejanos
    trendH = puntos_mas_lejanos["index"].tolist()
    trendH.append(candle)   
    
    return trendH, backeval

## Detect Break Out

In [12]:
def trendChannel(row,candle, backeval, window, sl_highs, interc_highs):

    if (candle-backeval-window)<0:
        return np.nan
   
    ini=candle-backeval
    fin=candle+window
    trendcurrhigh = np.where(np.logical_or((row.index > fin),  (row.index <  ini)), np.nan, (sl_highs*row.index + interc_highs))
    return trendcurrhigh

In [13]:
def isBreakOut(candleEval,backcandles, window, start, candle):
    if (candleEval-backcandles-window)<0:
        return 0

    if candleEval==start:
        prev_idx = candleEval
    else:
        prev_idx = candleEval-1
        
    prev_high = dfpl.loc[prev_idx].High
    prev_low = dfpl.loc[prev_idx].Low
    prev_close = dfpl.loc[prev_idx].Close
    
    curr_idx = candleEval
    curr_high = dfpl.loc[candleEval].High
    curr_low = dfpl.loc[candleEval].Low
    curr_close = dfpl.loc[candleEval].Close
    curr_open = dfpl.loc[candleEval].Open

    trend_prevhigh=dfpl.loc[prev_idx].trendcurrhigh
    trend_currhigh=dfpl.loc[candleEval].trendcurrhigh
    
    if ( prev_low < trend_prevhigh and
        prev_close > trend_prevhigh and
        curr_open > trend_currhigh and
        curr_close > trend_currhigh and
        candleEval>candle): #and r_sq_h > 0.9
        return 2

    elif (curr_open > trend_currhigh and
        curr_close > trend_currhigh and candleEval>candle):
        return 3
    
    else:
        return np.nan

In [14]:
def breakpointpos(x):
    if x['isBreakOutIni'] in (2,3):
        return x['Low']+3e-3
    elif x['isBreakOutIni']==1:
        return x['High']-3e-3
    else:
        return np.nan

##### Hallando casos de Ruptura del Canal Bajista

In [15]:
#Archivo anterior
ruta_archivo='data/cb_h.txt'
if os.path.exists(ruta_archivo):
    # Cargar el archivo
    df_casos2 = pd.read_csv(ruta_archivo, sep='\t')
    print("Archivo cargado correctamente.")
else:
    # Crear un DataFrame vacío
    df_casos2 = pd.DataFrame()
    print("Archivo no existe. Se creó un DataFrame vacío.")

Archivo cargado correctamente.


In [16]:
df_casos2["datetime"] = pd.to_datetime(df_casos2['datetime'])
df_casos2["datetime2"] = df_casos2['datetime'].dt.tz_localize(None)

In [17]:
df_casos = pd.DataFrame()
caso = 0
ticker2 = ""
window=5
backeval=75
backevalTrend=0
dfpl = pd.DataFrame
trendH = []
trendL = []
cant = 0
for i, row in df.iterrows():
    if(df['rcb01_01'][i]==1): #posibles casos 
        #print("posible caso:",i, df['companyName'][i])
        cant = 0
        candle = i
        cnt = 0
        valiniHigh = df.loc[candle,"High"]
        #valiniLow = df.loc[candle,"Low"]
        ticker = df['companyName'][i]
        #Reinicio de casos por company
        if ticker2 != ticker:
            ticker2 = ticker
            filtro = df_casos2[(df_casos2['datetime2'] <  fecha_menos_15) & (df_casos2['companyName']==ticker) & (df_casos2["ind_posicion"]==0)].copy()
            if filtro.shape[0]>0:
                caso = filtro["caso"].max()
            else:
                caso = 0
        dfpl = df[(df.companyName==ticker)].loc[i-backeval:i+backeval]                           
        h=1
        l=1
        cant = cant +1
        cantHorasTrend=0
        trendH, backevalTrend = backcandle(candle)        
        if (candle-backevalTrend>75):
            backevalTrend = candle-75
            
        dfpl['datetime'] = pd.to_datetime(dfpl['datetime'])
        cantHorasTrend = (dfpl.loc[candle,"datetime"] - dfpl.loc[backevalTrend,"datetime"]).total_seconds() / 3600

        if (cantHorasTrend>48): #Verificacion de caida por lo menos 2 dias
            backevalTrend2 = candle - backevalTrend
            #revision de trend negativo indica que esta cayendo, solo me quedo con estos casos
            sl_highs, interc_highs, r_sq_h = collect_channel(candle, backevalTrend2, trendH)
            dfpl.loc[trendH, "trendH"] = 1
        
            if (sl_highs<=0): #solo tendencias a la baja and ind_vela>0 sl_lows<0 and r_sq_h>=0.7  and sl_highs<=-0.1 and ind_vela>0
                #crear linea de tendencia high y low
                dfpl['trendcurrhigh']  = trendChannel(dfpl,candle,backevalTrend2,window,sl_highs,interc_highs)
                
                promVol = dfpl.loc[i-backeval:i]["Volume"].mean()

                dfpl["bearishSlope"] = np.nan #pendiente bajista en vacio
                dfpl.loc[i, "bearishSlope"] = 1 #pendiente bajista solo al punto evaluado            
                dfpl["ind_posicion"] = 0
                dfpl.loc[(dfpl.index < i), 'ind_posicion'] = -1
                dfpl.loc[(dfpl.index > i), 'ind_posicion'] = 1
                dfpl["promVol"] = promVol
                dfpl["sl_highs"] = sl_highs
                dfpl["r_sq_h"] = r_sq_h
        
                start = dfpl.index[0]
                dfpl["isBreakOut"] = [isBreakOut(candleEval, backevalTrend2, window, start, candle) for candleEval in dfpl.index]
                # Solo me quedo con el primer BREAK OUT
                cnt = dfpl.query("isBreakOut in (2,3) and Close>Open").shape[0] 
                id=0
                id2=0
                if cnt>0:
                    id = dfpl.query("isBreakOut in (2,3) and Close>Open").index[0]                        
                    #dfpl['isBreakOutIni'] = np.where(id!=dfpl.index, np.nan, dfpl['isBreakOut'])                        
                    dfpl.loc[id,'isBreakOutIni'] = 1
                    dfpl['breakpointpos'] = dfpl.apply(lambda row: breakpointpos(row), axis=1)
                    cnt2 = dfpl.query("index>@id and isPivot==1").shape[0]
                    
                    if cnt2>0:
                        id2 = dfpl.query("index>@id and isPivot==1").index[0]
                        dfpl.loc[id2,'isBreakOutFinal'] = 1
                    else:
                        dfpl['isBreakOutFinal'] = np.nan                      
                    
                else:
                    dfpl['isBreakOutIni'] = np.nan                        
                    dfpl['breakpointpos'] = np.nan
        
                if cnt>0:
                    #INSERT CASO
                    caso=caso+1                            
                    dfpl["caso"] = caso
                    print(i, df['companyName'][i])
                    if len(df_casos)<1:
                        df_casos = dfpl
                    else:
                        df_casos = pd.concat([df_casos, dfpl],ignore_index=False)


10479 AAPL


13954 AMZN
13966 AMZN


27990 TNA


31508 GLD
31535 GLD


35022 SLV


...

84232 PLTR


98712 MSTR


102238 UNH


105734 GOOG


112753 UBER


In [18]:
df_newCasos2 = pd.DataFrame()
for ticker in tickers:
    #Informacion anterior por ticker
   
    filtro = df_casos2[(df_casos2['datetime2'] <  fecha_menos_15) & (df_casos2['companyName']==ticker) & (df_casos2["ind_posicion"]==0)].copy()
    caso_max = filtro["caso"].max()
    print(caso_max)

    df_ticker = df_casos2[(df_casos2['caso'] <=  caso_max) & (df_casos2['companyName']==ticker)].copy()
    df_ticker = df_ticker.reset_index(drop=True)

    df_ticker = df_ticker.drop('datetime2', axis=1)

    print(df_ticker.shape[0])
    #df_ticker['datetime'] = pd.to_datetime(df_ticker['datetime'])

    #Informacion actual por ticker
    df_ticker2 = pd.DataFrame()
    if df_casos.shape[0]>0:
        df_ticker2 = df_casos[(df_casos['companyName']==ticker)]    
        df_ticker2 = df_ticker2.sort_values(by=['datetime'], ascending=[True])

     
    df_newCasos2 = pd.concat([df_newCasos2, df_ticker],ignore_index=True)
    df_newCasos2 = pd.concat([df_newCasos2, df_ticker2],ignore_index=True)


23
3458
35
5285
34
5134
33
4983
34
5134
35
5282
35
5285
32
4830
24


3624
32
4832


34
5134
26
3926
29
4379
30
4530
33
4983
31
4681
36
5436
25


3775
36
5428


38
5738
29
4379
31
4681
35
5285
36
5436
39
5889
44
6631


27
4077
2


295
38
5733
30
4530
29
4379
25
3762
43
6487


In [19]:
#path=r'D:\PROYECTO_CARLOS\Notebooks\notebooks\data\cb_h.txt'
path='data/cb_h.txt'
if os.path.exists(path):
    os.remove(path)
else:
    print("File does not exists. File needs to be created.")
with open(path, 'a') as f:
    df_newCasos2.to_csv(path, header=True, index=None, sep='\t', mode='w')

##### BACKTESTING
###### Usando paquete backtesting.py

In [20]:
df_casos2 = df_newCasos2.query("isBreakOutIni==1 or isBreakOutFinal==1").copy()

In [21]:
# =============== BACKTESTING ================= #
# obteniendo dataframe backtesting
df['datetime']=pd.to_datetime(df['datetime'])
df_backTesting = pd.merge(df, df_casos2[['datetime','companyName', 'isBreakOutIni','isBreakOutFinal','caso']], on = ['companyName','datetime'], how='left')
# datos de trade se asigna una vela anterior debido a que backtesting.py toma entra como la siguiente vela
df_backTesting['isBreakOutIni'] = df_backTesting['isBreakOutIni'].shift(-1)
df_backTesting['isBreakOutFinal'] = df_backTesting['isBreakOutFinal'].shift(-1)
#modificamos indice con el compo datetime
df_backTesting.set_index('datetime', inplace=True)

In [22]:
#Estrategia
class strategyRupturaCanal(Strategy):    
    def init(self):
        self.breakout_entry = self.I(lambda x: x, self.data.isBreakOutIni) #Indica la señal entrada al backtesting
        self.breakout_exit = self.I(lambda x: x, self.data.isBreakOutFinal) #Indica la señal salida al backtesting

    def next(self):       
        #print(self.data)        
        if self.breakout_entry[-1] == 1:
            self.buy()  #COMPRA
        elif self.breakout_exit[-1] == 1 and self.position:
            self.position.close() #CIERRE
        

In [23]:
# Por cada TIcket
my_stats = {}
estadisticas = pd.DataFrame()          # lista para ticker
tradesprev = pd.DataFrame()     # lista para cada caso por ticker
for ticker in df_backTesting['companyName'].unique():
    ticker_data = df_backTesting[df_backTesting['companyName'] == ticker].copy() #Filtro por cada ticker
    bt = Backtest(ticker_data, strategyRupturaCanal, cash=10_000)
    my_stats = bt.run()
    listTrades = my_stats['_trades']
    tradesdf = pd.DataFrame(listTrades) #trades
    tradesdf['Ticker'] = ticker
    stats_filter={
        'Ticker':ticker,
        'EntryTime':my_stats['Start'],
        'ExitTime':my_stats['End'],
        'Return [%]': my_stats['Return [%]'],
        'CAGR [%]': my_stats['Return (Ann.) [%]'],
        'Sharpe Ratio': my_stats['Sharpe Ratio'],
        'Max. Drawdown [%]': my_stats['Max. Drawdown [%]'],
        'Win Rate [%]': my_stats['Win Rate [%]'],
        '# Trades': my_stats['# Trades'],
        'Expectancy [%]': my_stats['Expectancy [%]'],
        'Profit Factor': my_stats['Profit Factor'],
        'Duration': my_stats['Duration'],
        'Avg. Trade [%]':my_stats['Avg. Trade [%]'],
        'Max. Trade Duration':my_stats['Max. Trade Duration'],
        'Avg. Trade Duration':my_stats['Avg. Trade Duration']
    }
    stat=pd.DataFrame([stats_filter])
    if tradesprev.shape[0]==0:
        tradesprev = tradesdf.copy()
        estadisticas=stat.copy()
    else:
        tradesprev = pd.concat([tradesprev, tradesdf],ignore_index=True)
        estadisticas=pd.concat([estadisticas, stat],ignore_index=True)
    # Mostramos
    #bt.plot(resample=False, filename=f"plot_BreakOUT_{ticker}")

Backtest.run:   0%|          | 0/3157 [00:00<?, ?bar/s]

/usr/share/miniconda/envs/test/lib/python3.10/site-packages/backtesting/backtesting.py:954: UserWarning: time=2050: Broker canceled the relative-sized order due to insufficient margin.
  warnings.warn(
/usr/share/miniconda/envs/test/lib/python3.10/site-packages/backtesting/backtesting.py:954: UserWarning: time=2068: Broker canceled the relative-sized order due to insufficient margin.
  warnings.warn(
/usr/share/miniconda/envs/test/lib/python3.10/site-packages/backtesting/backtesting.py:954: UserWarning: time=3067: Broker canceled the relative-sized order due to insufficient margin.
  warnings.warn(
                                                       

Backtest.run:   0%|          | 0/3409 [00:00<?, ?bar/s]

/usr/share/miniconda/envs/test/lib/python3.10/site-packages/backtesting/backtesting.py:954: UserWarning: time=399: Broker canceled the relative-sized order due to insufficient margin.
  warnings.warn(
/usr/share/miniconda/envs/test/lib/python3.10/site-packages/backtesting/backtesting.py:954: UserWarning: time=2349: Broker canceled the relative-sized order due to insufficient margin.
  warnings.warn(
/usr/share/miniconda/envs/test/lib/python3.10/site-packages/backtesting/backtesting.py:954: UserWarning: time=3070: Broker canceled the relative-sized order due to insufficient margin.
  warnings.warn(
                                                       

Backtest.run:   0%|          | 0/3353 [00:00<?, ?bar/s]

...

/usr/share/miniconda/envs/test/lib/python3.10/site-packages/backtesting/backtesting.py:954: UserWarning: time=1903: Broker canceled the relative-sized order due to insufficient margin.
  warnings.warn(


/usr/share/miniconda/envs/test/lib/python3.10/site-packages/backtesting/backtesting.py:954: UserWarning: time=2916: Broker canceled the relative-sized order due to insufficient margin.
  warnings.warn(
/usr/share/miniconda/envs/test/lib/python3.10/site-packages/backtesting/backtesting.py:954: UserWarning: time=3415: Broker canceled the relative-sized order due to insufficient margin.
  warnings.warn(
                                                       

Backtest.run:   0%|          | 0/3337 [00:00<?, ?bar/s]

/usr/share/miniconda/envs/test/lib/python3.10/site-packages/backtesting/backtesting.py:954: UserWarning: time=392: Broker canceled the relative-sized order due to insufficient margin.
  warnings.warn(
/usr/share/miniconda/envs/test/lib/python3.10/site-packages/backtesting/backtesting.py:954: UserWarning: time=825: Broker canceled the relative-sized order due to insufficient margin.
  warnings.warn(
/usr/share/miniconda/envs/test/lib/python3.10/site-packages/backtesting/backtesting.py:954: UserWarning: time=1564: Broker canceled the relative-sized order due to insufficient margin.
  warnings.warn(


/usr/share/miniconda/envs/test/lib/python3.10/site-packages/backtesting/backtesting.py:954: UserWarning: time=2517: Broker canceled the relative-sized order due to insufficient margin.
  warnings.warn(
/usr/share/miniconda/envs/test/lib/python3.10/site-packages/backtesting/backtesting.py:954: UserWarning: time=2533: Broker canceled the relative-sized order due to insufficient margin.
  warnings.warn(
/usr/share/miniconda/envs/test/lib/python3.10/site-packages/backtesting/backtesting.py:954: UserWarning: time=3424: Broker canceled the relative-sized order due to insufficient margin.
  warnings.warn(
                                                       

In [24]:
ini = df_casos2[(df_casos2['isBreakOutIni']==1)].copy()
df_casos3_ini=ini.rename(columns={'Open': 'EntryPrice', 'datetime': 'EntryTime', 'companyName': 'Ticker'})[['EntryPrice','Ticker','caso','EntryTime']]
final = df_casos2[(df_casos2['isBreakOutFinal']==1)].copy()
df_casos3_final=final.rename(columns={'Open': 'ExitPrice', 'datetime': 'ExitTime', 'companyName': 'Ticker'})[['ExitPrice','Ticker','caso','ExitTime']]

In [25]:
df_casos3=df_casos3_ini.merge(df_casos3_final,on=['Ticker','caso'],how='outer')

In [26]:
trades = pd.merge(df_casos3, tradesprev, on = ['EntryTime','Ticker'], how='left')

In [27]:
def exitPrice(x):
    if pd.isna(x['ExitPrice_x']):
        return x['ExitPrice_y']
    else:
        return x['ExitPrice_x']
trades['ExitPrice'] = trades.apply(lambda row: exitPrice(row), axis=1)

In [28]:
trades = trades.rename(columns={'EntryPrice_x': 'EntryPrice'})
trades = trades.drop(['EntryPrice_y','ExitPrice_x', 'ExitPrice_y'], axis=1)

In [29]:
def exitTime(x):
    if pd.isna(x['ExitTime_x']):
        return x['ExitTime_y']
    else:
        return x['ExitTime_x']
trades['ExitTime'] = trades.apply(lambda row: exitTime(row), axis=1)

In [30]:
def duration(x):
    if pd.isna(x['Duration']):
        if pd.isna(x['ExitTime']):
            return pd.NaT
        else:
            return x['ExitTime']-x['EntryTime']
    else:
        return x['Duration']
trades['Duration'] = trades.apply(lambda row: duration(row), axis=1)

In [31]:
trades = trades.drop(['ExitTime_x', 'ExitTime_y'], axis=1)

In [32]:
trades=trades[['Ticker','EntryTime','ExitTime','EntryPrice','ExitPrice','Duration','Size','EntryBar','ExitBar','ReturnPct','PnL','caso']]

In [33]:
for item in estadisticas['Ticker'].unique():
    count = trades[trades["Ticker"] == item].shape[0]
    estadisticas.loc[estadisticas['Ticker'] == item, '# Trades'] = count

In [34]:
#EXPORTACION DE DATAFRAME
path = 'data/backtesting'

# Asegurarse de que el directorio existe
os.makedirs(path, exist_ok=True)

path_trades = os.path.join(path, 'trades_cb.txt')
path_estadisticas = os.path.join(path, 'estadisticas_cb.txt')

for file_path in [path_trades, path_estadisticas]:
    if os.path.exists(file_path):
        os.remove(file_path)
    else:
        print(f"File does not exist: {file_path}. It will be created.")

trades.to_csv(path_trades, header=True, index=False, sep='\t', mode='w')
estadisticas.to_csv(path_estadisticas, header=True, index=False, sep='\t', mode='w')
